In [1]:
!nvidia-smi

Fri Mar 29 15:56:13 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro P5000        Off  | 00000000:00:05.0 Off |                  Off |
| 26%   33C    P8     6W / 180W |      2MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Ensemble Model
- Roberta Base Model for end part of text
- Bert Base Uncased Model for start part of text

In [2]:
import os
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split

from transformers import AutoTokenizer, AutoModel
from transformers import Trainer, TrainingArguments


from tqdm import tqdm


## Notebook Variables

In [3]:
# setting the model name
PRE_TRAINED_MODEL_NAME_1 = 'roberta-base'
PRE_TRAINED_MODEL_NAME_2 ='bert-base-uncased'

# setting the dataset
dataset='essays_cleaned.csv'

# setting the data path
if os.path.exists(f'/datasets/essays/{dataset}'):
    DATAPATH=f'/datasets/essays/{dataset}'
else:
    DATAPATH=f'../data/{dataset}'

# setting the checkpoint path 
if os.path.exists('ckpts'):
    CHECKPOINTPATH = 'ckpts/Persnality_trait'
else:
    CHECKPOINTPATH = '../checkpoints/Persnality_traits'

# training parameters
MAX_LEN = 512
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
TEST_BATCH_SIZE = 16
EPOCHS = 10
THRESHOLD = 0.5 # threshold for the sigmoid function

# TOKENIZER
tokenizer_1 = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME_1)
tokenizer_2 = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME_2)

# setting the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
DATAPATH,CHECKPOINTPATH

('/datasets/essays/essays_cleaned.csv', 'ckpts/Persnality_trait')

### Loading dataset

In [5]:
# loading the data
df = pd.read_csv(DATAPATH,encoding="utf-8")

In [6]:
# create a dataset class
class EssayDataset(Dataset):
    def __init__(self, df,target_list, max_len=MAX_LEN,tokenizer1=tokenizer_1, tokenizer2=tokenizer_2):
        self.df = df
        self.max_len = max_len
        self.text = list(df['TEXT'])
        self.tokenizer1 = tokenizer1
        self.tokenizer2=tokenizer2
        self.targets = self.df[target_list].values
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.text[idx]
        textRight=text
        if(len(text)>self.max_len):
            textRight=textRight[:self.max_len]
        inputs1 = self.tokenizer1.encode_plus(
            textRight,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_token_type_ids=True,
            return_tensors='pt'
          
        )
        # bert tokenizer
        inputs2 = self.tokenizer2.encode_plus(
            text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_token_type_ids=True,
            return_tensors='pt'
        )
        return {
            'text': text,
            'input_ids_roberta': inputs1['input_ids'].flatten(),
            'attention_mask_roberta': inputs1['attention_mask'].flatten(),
            'token_type_ids_roberta': inputs1['token_type_ids'].flatten(),
            'input_ids_bert': inputs2['input_ids'].flatten(),
            'attention_mask_bert': inputs2['attention_mask'].flatten(),
            'token_type_ids_bert': inputs2['token_type_ids'].flatten(),
            'targets': torch.FloatTensor(self.targets[idx])
        }

In [7]:
# get the targets list
target_list = list(df.columns[1:])
print(f'Target list: {target_list}')
# create the datasets
dataset= EssayDataset(df,target_list)

# split data
train_size = int(0.8 * len(df))
test_size = int(0.1 * len(df))
val_size = len(df) - train_size - test_size

train_dataset, test_dataset, val_dataset = random_split(dataset, [train_size, test_size, val_size])

# create the dataloaders
train_data_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
test_data_loader = DataLoader(test_dataset, batch_size=TEST_BATCH_SIZE, shuffle=True)
val_data_loader = DataLoader(val_dataset, batch_size=VALID_BATCH_SIZE, shuffle=True)


Target list: ['cEXT', 'cNEU', 'cAGR', 'cCON', 'cOPN']


In [8]:
batch=next(iter(train_data_loader))
print('Input ids shape:',batch['input_ids_roberta'].shape, 'Attention mask shape:',batch['attention_mask_roberta'].shape, 'Token type ids shape:',batch['token_type_ids_roberta'].shape, 'Targets shape:',batch['targets'].shape)
batch

Input ids shape: torch.Size([16, 512]) Attention mask shape: torch.Size([16, 512]) Token type ids shape: torch.Size([16, 512]) Targets shape: torch.Size([16, 5])


{'text': ['     Hey well I am really very nervous writing down all of my thoughts and feelings on this computer and not in my own personal journal Because atleast there I know I will be the only one reading it But I guess I ll start writing about what I really have on my mind My boyfriend and best friend just took off on a long truck ride back home which is about six and half hours away They Jeremiah and Lauren came to visit this weekend and even though they have only been gone for an hour and fifteen minutes I miss them like crazy Jeremiah means everything to me and I cannot stand the thought of him being five hundred miles away But then again I decided to come to Texas and not to Tech Right Well that is one thing that is bothering me the other thing eating away at me is that my new stepmother who is only four years older than me is causing some major problems back at home And on that note it really ticks me off that as soon as I left home she has been on a rampage and really screwing

### Models 

In [9]:
class BERTClass(torch.nn.Module):
    def __init__(self, PRE_TRAINED_MODEL_NAME, num_classes=5, dropout=0.3):
        super(BERTClass, self).__init__()
        self.bert_model = AutoModel.from_pretrained(PRE_TRAINED_MODEL_NAME, return_dict=True, output_attentions=True)
        self.dropout = torch.nn.Dropout(dropout)
        self.linear = torch.nn.Linear(768, num_classes)

    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids, 
            attention_mask=attn_mask, 
            token_type_ids=token_type_ids
        )
        attentions=output.attentions
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output, attentions

### Training

In [10]:
import tqdm.notebook as tq
# Training of the model for one epoch

def train_model(training_loader, model, optimizer,loss_fn, scheduler=None, device=device, model_name='bert'):

    losses = []
    correct_predictions = 0
    num_samples = 0
    # set model to training mode (activate droput, batch norm)
    model.train()
    # initialize the progress bar
    loop = tq.tqdm(enumerate(training_loader), total=len(training_loader), 
                      leave=True, colour='steelblue')
    for _, data in loop:
        ids = data[f'input_ids_{model_name}'].to(device, dtype = torch.long)
        mask = data[f'attention_mask_{model_name}'].to(device, dtype = torch.long)
        token_type_ids = data[f'token_type_ids_{model_name}'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)
        
        # forward
        outputs,_ = model(ids, mask, token_type_ids) # (batch,predict)=(32,8)
        ids = ids.cpu().detach().numpy()
        mask = mask.cpu().detach().numpy()
        loss = loss_fn(outputs, targets)
        losses.append(loss.item())
        # training accuracy, apply sigmoid, round (apply thresh 0.5)
        outputs = torch.sigmoid(outputs).cpu().detach().numpy().round()
        targets = targets.cpu().detach().numpy()
        correct_predictions += np.sum(outputs==targets)
        num_samples += targets.size   # total number of elements in the 2D array
       
        # backward
        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        # grad descent step
        optimizer.step()

    # returning: trained model, model accuracy, mean loss
    return model, float(correct_predictions)/num_samples, np.mean(losses)


In [11]:
# evaluation of the model

def eval_model(validation_loader, model,loss_fn,model_name='bert'):
    losses = []
    correct_predictions = 0
    num_samples = 0
    # set model to eval mode (turn off dropout, fix batch norm)
    model.eval()

    with torch.no_grad():
        for btch_idx, data in enumerate(validation_loader, 0):
            ids = data[f'input_ids_{model_name}'].to(device, dtype = torch.long)
            mask = data[f'attention_mask_{model_name}'].to(device, dtype = torch.long)
            token_type_ids = data[f'token_type_ids_{model_name}'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs,_ = model(ids, mask, token_type_ids)

            loss = loss_fn(outputs, targets)
            losses.append(loss.item())

            # validation accuracy
            # add sigmoid, for the training sigmoid is in BCEWithLogitsLoss
            outputs = torch.sigmoid(outputs).cpu().detach().numpy().round()
            targets = targets.cpu().detach().numpy()
            correct_predictions += np.sum(outputs==targets)
            num_samples += targets.size   # total number of elements in the 2D array
    return float(correct_predictions)/num_samples, np.mean(losses)

In [12]:
# BCEWithLogitsLoss combines a Sigmoid layer and the BCELoss in one single class. 
# This version is more numerically stable than using a plain Sigmoid followed 
# by a BCELoss as, by combining the operations into one layer, 
# we take advantage of the log-sum-exp trick for numerical stability.
def loss_fn(preds, labels):
    return torch.nn.BCEWithLogitsLoss()(preds, labels) 

In [13]:
# training the model
from collections import defaultdict
def train(PRE_TRAINED_MODEL_NAME,model_name):
    model=BERTClass(PRE_TRAINED_MODEL_NAME)
    model.to(device)

    # setting the optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)    

    history = defaultdict(list)
    best_accuracy = 0
    print(f'{PRE_TRAINED_MODEL_NAME}')
    for epoch in range(1, EPOCHS+1):
        print(f'Epoch {epoch}/{EPOCHS}')
        model, train_acc, train_loss = train_model(train_data_loader, model, optimizer,loss_fn=loss_fn,model_name=model_name)
        val_acc, val_loss = eval_model(val_data_loader, model,loss_fn=loss_fn,model_name=model_name)

        print(f'train_loss={train_loss:.4f}, val_loss={val_loss:.4f} train_acc={train_acc:.4f}, val_acc={val_acc:.4f}')

        history['train_acc'].append(train_acc)
        history['train_loss'].append(train_loss)
        history['val_acc'].append(val_acc)
        history['val_loss'].append(val_loss)
        # save the best model
        if val_acc > best_accuracy:
            torch.save(model.state_dict(),CHECKPOINTPATH+f'_{PRE_TRAINED_MODEL_NAME}.bin')
            best_accuracy = val_acc

    print(f'{PRE_TRAINED_MODEL_NAME} Best accuracy: {best_accuracy}')

In [14]:
# train both models
train(PRE_TRAINED_MODEL_NAME_1,"roberta")


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


roberta-base
Epoch 1/10


  0%|          | 0/124 [00:00<?, ?it/s]

train_loss=0.6934, val_loss=0.6919 train_acc=0.5068, val_acc=0.5242
Epoch 2/10


  0%|          | 0/124 [00:00<?, ?it/s]

...

train_loss=0.4904, val_loss=0.7689 train_acc=0.7727, val_acc=0.5532
Epoch 9/10


  0%|          | 0/124 [00:00<?, ?it/s]

train_loss=0.4334, val_loss=0.8258 train_acc=0.8070, val_acc=0.5573
Epoch 10/10


  0%|          | 0/124 [00:00<?, ?it/s]

train_loss=0.3820, val_loss=0.9159 train_acc=0.8363, val_acc=0.5492
roberta-base Best accuracy: 0.5935483870967742


In [15]:
import gc         # garbage collect library
gc.collect()
torch.cuda.empty_cache() 

In [16]:
train(PRE_TRAINED_MODEL_NAME_2,"bert")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


bert-base-uncased
Epoch 1/10


  0%|          | 0/124 [00:00<?, ?it/s]

train_loss=0.6973, val_loss=0.6856 train_acc=0.5128, val_acc=0.5548
Epoch 2/10


  0%|          | 0/124 [00:00<?, ?it/s]

...

train_loss=0.5516, val_loss=0.6903 train_acc=0.7293, val_acc=0.5903
Epoch 9/10


  0%|          | 0/124 [00:00<?, ?it/s]

train_loss=0.5108, val_loss=0.7157 train_acc=0.7625, val_acc=0.5847
Epoch 10/10


  0%|          | 0/124 [00:00<?, ?it/s]

train_loss=0.4635, val_loss=0.7611 train_acc=0.7926, val_acc=0.5677
bert-base-uncased Best accuracy: 0.5903225806451613


### Testing the models

In [17]:
# get the predictions
def get_predictions(model,data_loader,model_name):
    model.eval()
    text=[]
    predictions = []
    predictions_probs = []
    real_values = []
    with torch.no_grad():
        for data in data_loader:
            text.extend(data['text'])
            ids = data[f'input_ids_{model_name}'].to(device, dtype = torch.long)
            mask = data[f'attention_mask_{model_name}'].to(device, dtype = torch.long)
            token_type_ids = data[f'token_type_ids_{model_name}'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs,_ = model(ids, mask, token_type_ids)
            outputs = torch.sigmoid(outputs)
            predictions.extend(outputs.cpu().detach().round())
            real_values.extend(targets.cpu().detach())
            predictions_probs.extend(outputs.cpu().detach())      
    predictions = torch.stack(predictions)
    predictions_probs = torch.stack(predictions_probs)
    real_values = torch.stack(real_values)
    return text,predictions,predictions_probs, real_values

In [18]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [19]:
def get_single_metrics(model,test_data_loader,model_name):
    # get predictions for the test data
    text, predictions, prediction_probs, real_values = get_predictions(model, test_data_loader,model_name)
    accuracy = accuracy_score(real_values.view(-1), predictions.view(-1))
    accuracy_scores = {}
    for i in range(len(target_list)):
        accuracy_scores[target_list[i]] = accuracy_score(real_values[:,i], predictions[:,i])
    report=classification_report(real_values, predictions, target_names=target_list)
    print(f"Accuracy {accuracy}")
    print(f'Accuracy Scores {accuracy_scores}')
    print(f"classification_report{report}")
    return accuracy,accuracy_score,report

In [20]:
# load roberta
model_roberta=BERTClass(PRE_TRAINED_MODEL_NAME_1)
model_roberta.load_state_dict(torch.load(CHECKPOINTPATH+f'_{PRE_TRAINED_MODEL_NAME_1}.bin'))
model_roberta.eval()
model_roberta.to(device)

# load bert
model_bert=BERTClass(PRE_TRAINED_MODEL_NAME_2)
model_bert.load_state_dict(torch.load(CHECKPOINTPATH+f'_{PRE_TRAINED_MODEL_NAME_2}.bin'))
model_bert.eval()
model_bert.to(device)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions

BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [21]:
# get metrics for both models
get_single_metrics(model_roberta,test_data_loader,"roberta")
get_single_metrics(model_bert,test_data_loader,"bert")

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy 0.5609756097560976
Accuracy Scores {'cEXT': 0.5447154471544715, 'cNEU': 0.556910569105691, 'cAGR': 0.5203252032520326, 'cCON': 0.5772357723577236, 'cOPN': 0.6056910569105691}
classification_report              precision    recall  f1-score   support

        cEXT       0.60      0.69      0.64       144
        cNEU       0.55      0.64      0.59       123
        cAGR       0.53      0.58      0.55       126
        cCON       0.60      0.53      0.56       126
        cOPN       0.62      0.50      0.56       121

   micro avg       0.58      0.59      0.58       640
   macro avg       0.58      0.59      0.58       640
weighted avg       0.58      0.59      0.58       640
 samples avg       0.58      0.60      0.54       640

Accuracy 0.5552845528455285
Accuracy Scores {'cEXT': 0.5487804878048781, 'cNEU': 0.5609756097560976, 'cAGR': 0.5284552845528455, 'cCON': 0.556910569105691, 'cOPN': 0.5813008130081301}
classification_report              precision    recall  f1-score   s

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.5552845528455285,
 <function sklearn.metrics._classification.accuracy_score(y_true, y_pred, *, normalize=True, sample_weight=None)>,
 '              precision    recall  f1-score   support\n\n        cEXT       0.60      0.69      0.64       144\n        cNEU       0.56      0.54      0.55       123\n        cAGR       0.54      0.58      0.56       126\n        cCON       0.57      0.57      0.57       126\n        cOPN       0.55      0.78      0.65       121\n\n   micro avg       0.56      0.63      0.60       640\n   macro avg       0.56      0.63      0.59       640\nweighted avg       0.56      0.63      0.60       640\n samples avg       0.58      0.64      0.56       640\n')

In [25]:
_, _, prediction_probs, real_values = get_predictions(model_roberta, test_data_loader,"roberta")
_, _, prediction_probs, real_values = get_predictions(model_bert, test_data_loader,"bert")

weight1 = 1  # Weight for model 1
weight2 = 1  # Weight for model 2

# Combine probabilities using weighted averaging
combined_probs = (weight1 * prediction_probs + weight2 * prediction_probs) / (weight1 + weight2)
print(combined_probs)
predictions= combined_probs.round()
print(predictions)

tensor([[0.4565, 0.5897, 0.2768, 0.3034, 0.9305],
        [0.7815, 0.2013, 0.8386, 0.7884, 0.1424],
        [0.6110, 0.3704, 0.6355, 0.5999, 0.7785],
        ...,
        [0.7636, 0.6614, 0.4687, 0.5767, 0.7396],
        [0.3764, 0.4350, 0.6789, 0.7481, 0.7647],
        [0.6636, 0.3018, 0.8050, 0.7310, 0.0875]])
tensor([[0., 1., 0., 0., 1.],
        [1., 0., 1., 1., 0.],
        [1., 0., 1., 1., 1.],
        ...,
        [1., 1., 0., 1., 1.],
        [0., 0., 1., 1., 1.],
        [1., 0., 1., 1., 0.]])


In [28]:
# ensemble metrics
accuracy = accuracy_score(real_values.view(-1), predictions.view(-1))
accuracy_scores = {}
for i in range(len(target_list)):
    accuracy_scores[target_list[i]] = accuracy_score(real_values[:,i], predictions[:,i])
# classification_report=classification_report(real_values, predictions, target_names=target_list)
print(f"Accuracy {accuracy}")
print(f'Accuracy Scores {accuracy_scores}')
# print(f"classification_report{classification_report}")

Accuracy 0.5552845528455285
Accuracy Scores {'cEXT': 0.5487804878048781, 'cNEU': 0.5609756097560976, 'cAGR': 0.5284552845528455, 'cCON': 0.556910569105691, 'cOPN': 0.5813008130081301}
